# **<font color='crimson'>Алгоритм получения желаемых метрик A/B-теста для односторонней гипотезы, проверяющего уровень удовлетворенности клиентов услугами сервиса</font>**

---

**Выполнил**: Юмаев Егор

---

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

Компания, предоставляющая сервисные услуги, столкнулась с увеличением жалоб клиентов. По итогам ранее проведенных исследований было установлено, что уровень удовлетворенности клиентов составляет 80%. Увеличение числа негативных отзывов может свидетельствовать о снижении уровня удовлетворенности клиентов.

Сформулировано две гипотезы:

* **Н0**: уровень удовлетвореннотси клиентов равен 80%

* **Н1**: уровень удовлетворенности клиентов составляет менее 80%

Проверяемая гипотеза является **односторонней**: принципиально важно зафиксировать, если уровень удовлетворенности клиентов услугами сервиса снизился. Если уровень удовлетворенности вырос, это не представляет угрозы для будущего развития компании.

**Цель** состоит в том, чтобы на этапе **планирования** A/B-теста разработать процедуру, позволяющую сопоставить желаемые метрики теста и размер выборки.

Зафиксируем желаемые значения метрик A/B-теста:

* **MDE**: 10%

* **Power (1 - beta)**: 80%

* **Significance (alpha)**: 5%

Требуется подобрать:

* **размер выборки**

* **отклонение от нормы**

Нижеследующая часть отчета не содержит все итерации, пройденные для поиска размера выборки и порогового значения отклонения от нормы. Представлен конечный результат исследования соотношения желаемых значений метрик A/B теста и необходимого размера выборки.

## <font color='green'>**1 Подбор False Positive Rate (Significance)**</font>

---

In [2]:
# зададим размер выборки
sample_size = 113

Поскольку нулевая гипотеза предполагает, что уровень удовлетворенности клиентов равен 80%, именно такой уровень вероятности необходимо задать в функции np.random.binomial(): 0.8.

In [3]:
# для расчета False Positive Rate
# моделируем 100_000 экспериментов
n = 100_000
result = []

for i in tqdm(range(n)):
    np.random.seed(i) # воспроизводимость результатов
    result.append(np.random.binomial(sample_size, 0.8))

  0%|          | 0/100000 [00:00<?, ?it/s]

In [4]:
# сохраняем результаты в датафрейм
df = pd.DataFrame(result)

In [5]:
# выведем первые пять строк датафрейма с результатами
df.head()

,0
0,90
1,91
2,91
3,90
4,82


При вычислении отклонений для **односторонней** гипотезы (односторонней проверки) взятие модуля от результата разницы **не** применяется, в отличие от **двусторонних** гипотез.

Отклонения в большую сторону нас не интересуют. Необходимо "отловить" только отклонения в меньшую сторону.

In [6]:
# вычисляем отклонение для каждого эксперимента
df['deviation'] = df[0] - sample_size * 0.8

In [7]:
df.head()

,0,deviation
0,90,-0.4
1,91,0.6
2,91,0.6
3,90,-0.4
4,82,-8.4


Интерпретировать результат можно следующим образом:

* по первой строке: обзвонили 113 клиентов, довольны из них 90. Идеальное значение в 80% довольных клиентов не достигнуто

* по творой строке: обзвонили 113 клиентов, довольны из них 91. Идеальное значение в 80% довольных клиентов не только достигнуто, но перевыполнено

* и т.д.

In [8]:
# определим пороговый уровень для расчета False Positive Rate (Significance);
# порог подбирается "механически" для достижения
# требуемого уровня значимости
threshold = sample_size * 0.06
threshold

6.779999999999999

В отличие от **двустороннего** теста, где проверяется, что текущее значение >= threshold, для одностороннего теста, проверяющего, что значение может быть меньше порогового, следует проверить, что текущее значение <= - threshold.

In [9]:
# вычислим процент ситуаций, когда отклонение от sample_size * 0.8
# меньше или равно - threshold
(df['deviation'] <= - threshold).mean()

0.05642

## <font color='green'>**2 Подбор True Positive Rate и MDE**</font>

---

Порог отклонения зададим равным значению, подобранному в предыдущем пункте: **threshold**.

Т.к. гипотеза односторонняя (**Н1**: уровень удовлетворенности меньше 80%) при вызове функции np.random.binomial() из значения исходной вероятности 0.8 величину MDE будем вычитать, а не прибавлять, как в случае с двусторонней гипотезой.

In [10]:
# моделируем 100_000 экспериментов
n = 100_000
mde = 0.1
result = []

for i in tqdm(range(n)):
    np.random.seed(i) # воспроизводимость результатов
    result.append(np.random.binomial(sample_size, 0.8 - mde))

  0%|          | 0/100000 [00:00<?, ?it/s]

In [11]:
# сохраняем результаты в датафрейм
df = pd.DataFrame(result)

In [12]:
# выведем первые пять строк результата
df.head()

,0
0,78
1,81
2,85
3,77
4,76


In [13]:
# вычисляем отклонение для каждого эксперимента
df['deviation'] = df[0] - sample_size * 0.8

In [14]:
# выведем первые пять строк датасета df
df.head()

,0,deviation
0,78,-12.4
1,81,-9.4
2,85,-5.4
3,77,-13.4
4,76,-14.4


In [15]:
# вычислим процент ситуаций, когда отклонение от sample_size * 0.8
# больше или равно threshold
(df['deviation'] <= - threshold).mean()

0.81388

# **<font color='royalblue'>Выводы</font>**

---

(1) С целью подбора желаемых метрик A/B-теста для **односторонней** гипотезы были зафиксированы следующие значения метрик:

* **MDE**: 10%

* **Power (1 - beta)**: 80%

* **Significance (alpha)**: 5%


(2) В результате проведения ряда итераций подобраны:

* **размер выборки**

* **отклонение от нормы**


(3) **Отклонение от нормы** (пороговое значение) на каждой итерации корректировалось таким образом, чтобы значение **Significance (alpha)** было максимально приближено к 0.05 (5%).

(4) Основной упор был сделан на подбор наилучшего **размера выборки**: по итогам первой итерации первоначально выбранный размер (200 пользователей) оказался чрезмерным, т.к. обеспечил **Power (1 - beta)** в 95% (на 15% выше желаемого уровня). Поскольку именно размер выборки определяет стоимость проведения A\B-теста, на последующих итерациях размер выборки последовательно корректировался, пока не был найден оптимальный его размер: 113 пользователей.

(5) Размер выборки в 113 пользователей позволил на этапе планирования A/B-теста получить следующие прогнозные значения метрик:

* **MDE**: 10%

* **Power (1 - beta)**: 81.4%

* **Significance (alpha)**: 5.6%

Вычисленные размеры метрик A/B-теста наиболее близки к желаемым при размере выборки в 113 пользователей.


(6) В отчете представлены расчеты, полученные по итогам финальной итерации. Все промежуточные итерации в отчет **не** включены.